In [1]:
import holidays
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

In [2]:
file_path = '/Users/mymac/Google Drive/My Drive/Forex_Robot/'

In [3]:
# ----------------------------------------------------------------------------------------------------
# Get the data
# ----------------------------------------------------------------------------------------------------
def percentage_to_float(x):
  return float(x.strip('%')) / 100

us_holidays = holidays.US()

df = pd.read_csv(file_path + 'chris.csv')
df.columns = map(str.lower, df.columns)
df.drop(['simple date', 'day', 'day type name', 'holiday flag', 'date.1'], axis=1, inplace=True)
df.date = pd.to_datetime(df.date)
df[['beob', 'buob', 'bull hammer', 'bear hammer', 'bullish idf', 'bearish idf', 'bull pinbar', 'bear pinbar']] = df[['beob', 'buob', 'bull hammer', 'bear hammer', 'bullish idf', 'bearish idf', 'bull pinbar', 'bear pinbar']].fillna(0)
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)
df['% distance of 5ema to 50ema'] = df['% distance of 5ema to 50ema'].astype(str)
df['% distance of 5ema to 50ema'] = df['% distance of 5ema to 50ema'].apply(lambda x: percentage_to_float(x))
df['% distance of 5ema to 100ema'] = df['% distance of 5ema to 100ema'].astype(str)
df['% distance of 5ema to 100ema'] = df['% distance of 5ema to 100ema'].apply(lambda x: percentage_to_float(x))
df['% distance of 50ema to 100ema'] = df['% distance of 50ema to 100ema'].astype(str)
df['% distance of 50ema to 100ema'] = df['% distance of 50ema to 100ema'].apply(lambda x: percentage_to_float(x))
df['% gain on the day'] = df['% gain on the day'].astype(str)
df['% gain on the day'] = df['% gain on the day'].apply(lambda x: percentage_to_float(x))
df['range as a % of price'] = df['range as a % of price'].astype(str)
df['range as a % of price'] = df['range as a % of price'].apply(lambda x: percentage_to_float(x))
df['20 day rel vol %'] = df['20 day rel vol %'].astype(str)
df['20 day rel vol %'] = df['20 day rel vol %'].apply(lambda x: percentage_to_float(x))
daytype_onehot = pd.get_dummies(df.daytype, prefix='daytype')
df.drop('daytype', axis=1, inplace=True)
df = pd.concat([df, daytype_onehot], axis=1)
df['holiday'] = df['date'].apply(lambda x: 1 if x.strftime('%Y-%m-%d') in us_holidays else 0)
df['sin_day'] = np.sin(2 * np.pi * df['date'].dt.day / 7)
df['cos_day'] = np.cos(2 * np.pi * df['date'].dt.day / 7)
df['sin_month'] = np.sin(2 * np.pi * df['date'].dt.month / 12)
df['cos_month'] = np.cos(2 * np.pi * df['date'].dt.month / 12)

In [4]:
# ----------------------------------------------------------------------------------------------------
# Create targets
# ----------------------------------------------------------------------------------------------------
def create_target(dataset, i, pred_period=1):
  risk = 175 / 4
  trade = None

  on_open = df.loc[df.index[i], 'on open']
  rth_open = df.loc[df.index[i - 1], 'rth open']
  rth_close = df.loc[df.index[i - 1], 'close / rth close']
  ema_5, ema_50, ema_100 = df.loc[df.index[i - 1], ['5 ema', '50 ema', '100 ema']]
  vix = df.loc[df.index[i - 1], 'vix adj close']

  down_day = rth_close < rth_open
  ema_signal = ema_5 < ema_50 or ema_5 < ema_100
  vix_signal = vix >= 16

  trade_signal = down_day and ema_signal and vix_signal

  if trade_signal and trade is None:
    open_price = on_open
    stop_loss = on_open + risk

    trade = {'open_price': open_price, 'stop_loss': stop_loss, 'trade_type': 'sell'}

  if trade is not None:
    on_high = df.loc[df.index[i], 'on hi']
    on_low = df.loc[df.index[i], 'on lo']
    on_close = df.loc[df.index[i], 'rth open']

    if trade['trade_type'] == 'buy':
      if on_low <= trade['stop_loss']:
        return 0

      else:
        trade_amount = on_close - trade['open_price']

        return 1 if trade_amount > 0 else 0

    else:
      if on_high >= trade['stop_loss']:
        return 0

      else:
        trade_amount = trade['open_price'] - on_close

        return 2 if trade_amount > 0 else 0

  return 0

targets = [create_target(df, i) for i in range(len(df))]
df['target'] = targets
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

print(df['target'].value_counts())

0    2236
2     122
Name: target, dtype: int64


In [5]:
# ----------------------------------------------------------------------------------------------------
# Get the dates
# ----------------------------------------------------------------------------------------------------
dates = df['date']
df.drop('date', axis=1, inplace=True)

In [6]:
x = df.loc[:, df.columns != 'target']
y = df['target']

test_percentage = 0.3
cutoff_index = int(len(df) * (1 - test_percentage))

x_train_org = x.iloc[0:cutoff_index, :]
x_test_org = x.iloc[cutoff_index:, :]
y_train = y.iloc[0:cutoff_index, ]
y_test = y.iloc[cutoff_index:, ]

# Scale the input data
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train_org)
x_test = scaler.transform(x_test_org)

In [7]:
# ----------------------------------------------------------------------------------------------------
# Train the svm
# ----------------------------------------------------------------------------------------------------
svm = SVC(kernel='linear', gamma=10, class_weight='balanced', C=1000)
svm.fit(x_train, y_train)

y_pred = svm.predict(x_test)
print("Accuracy: ", metrics.accuracy_score(y_test, y_pred))
print("Balanced accuracy: ", metrics.balanced_accuracy_score(y_test, y_pred))

Accuracy:  0.8898305084745762
Balanced accuracy:  0.7590306055431499


In [8]:
# Run simulation
reward = 0
n_wins = 0
n_losses = 0
win_streak = 0
loss_streak = 0
curr_win_streak = 0
curr_loss_streak = 0
n_buys = 0
n_sells = 0
n_ticks = 0
trade = None
risk = 175 / 4

for i in range(cutoff_index, df.shape[0]):
  on_open = df.loc[df.index[i], 'on open']
  rth_open = df.loc[df.index[i - 1], 'rth open']
  rth_close = df.loc[df.index[i - 1], 'close / rth close']
  ema_5, ema_50, ema_100 = df.loc[df.index[i - 1], ['5 ema', '50 ema', '100 ema']]
  vix = df.loc[df.index[i - 1], 'vix adj close']

  down_day = rth_close < rth_open
  ema_signal = ema_5 < ema_50 or ema_5 < ema_100
  vix_signal = vix >= 16

  trade_signal = down_day and ema_signal and vix_signal
  curr_tup = np.array(df.loc[df.index[i], df.columns != 'target']).reshape(1, -1)
  pred = svm.predict(curr_tup)[0]
  pred_signal = pred == 2

  if trade_signal and pred_signal and trade is None:
    open_price = on_open
    # stop_loss = on_open - risk
    # stop_gain = on_open + profit
    stop_loss = on_open + risk

    trade = {'open_price': open_price, 'stop_loss': stop_loss, 'trade_type': 'sell'}

    n_sells += 1

  if trade is not None:
    on_high = df.loc[df.index[i], 'on hi']
    on_low = df.loc[df.index[i], 'on lo']
    on_close = df.loc[df.index[i], 'rth open']

    if trade['trade_type'] == 'buy':
      if on_low <= trade['stop_loss']:
        trade_amount = trade['stop_loss'] - trade['open_price']
        reward += trade_amount
        n_ticks += trade_amount * 4

        n_wins += 1 if trade_amount > 0 else 0
        n_losses += 1 if trade_amount < 0 else 0

        trade = None

        continue

      else:
        trade_amount = on_close - trade['open_price']
        reward += trade_amount
        n_ticks += trade_amount * 4

        n_wins += 1 if trade_amount > 0 else 0
        n_losses += 1 if trade_amount < 0 else 0

        trade = None

        continue

    else:
      if on_high >= trade['stop_loss']:
        trade_amount = trade['open_price'] - trade['stop_loss']
        reward += trade_amount
        n_ticks += trade_amount * 4

        n_wins += 1 if trade_amount > 0 else 0
        n_losses += 1 if trade_amount < 0 else 0

        trade = None

        continue

      else:
        trade_amount = trade['open_price'] - on_close
        reward += trade_amount
        n_ticks += trade_amount * 4

        n_wins += 1 if trade_amount > 0 else 0
        n_losses += 1 if trade_amount < 0 else 0

        trade = None

        continue

In [9]:
print(f'Reward: {reward}\nTicks: {n_ticks}\nBuys: {n_buys}\nSells: {n_sells}\nTrades: {n_buys + n_sells}\nWins: {n_wins}\nLosses: {n_losses}\nWin rate: {n_wins / (n_wins + n_losses)}')

Reward: -35.75
Ticks: -143.0
Buys: 0
Sells: 96
Trades: 96
Wins: 46
Losses: 50
Win rate: 0.4791666666666667
